In [1]:
import setup
setup.init()

In [2]:
from mypermit import permit_client


permit = permit_client

In [3]:
await permit.api.users.list()

PaginatedResultUserRead(data=[], total_count=0, page_count=0)

In [5]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [11]:
for u in User.objects.all():
    user_data = {
        "key": f"{u.id}",
        "first_name": f"{u.username}",
    }
    response = await permit.api.users.sync(user_data)
    print(response)



key='1' id=UUID('6b40ddfa-306b-4f68-9d83-c1cdf3ebaf1c') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') associated_tenants=[] roles=[] created_at=datetime.datetime(2025, 7, 22, 5, 26, 41, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 22, 5, 30, 43, tzinfo=datetime.timezone.utc) email=None first_name='soumo' last_name=None attributes=None
key='2' id=UUID('aaed34be-2e28-425f-b281-f6c03334e60b') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') associated_tenants=[] roles=[] created_at=datetime.datetime(2025, 7, 22, 5, 26, 41, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 22, 5, 30, 43, tzinfo=datetime.timezone.utc) email=None first_name='staff' last_name=None attributes=None
key='3' id=UUID('e

In [3]:
resources = [
    {
        "name": "Documents",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [ ]:
# read_only_permissions = [
#     "document:read",
#     "movie_disovery:search",
#     "movie_disovery:detail",
# ]


In [7]:
roles= [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": {
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail",
        }
    },
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": read_only_permissions,
    }
]